In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/eval-dataset/Eval_Dataset.xlsx
/kaggle/input/banglanews24/banglanews24.parquet


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import re,json,nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from tensorflow.keras.preprocessing.text import Tokenizer

data = pd.read_parquet('/kaggle/input/banglanews24/banglanews24.parquet')
data.head(10)

2024-10-01 15:40:50.644773: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 15:40:50.644919: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 15:40:50.801632: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,Text,Timestamp,URL
0,চাঁদপুর: প্রাচীন বন্দর নগরী ও তিন নদীর মোহনায় ...,"আপডেট: ২১৪৪ ঘণ্টা, নভেম্বর ১৯, ২০২৩",https://www.banglanews24.com/national/news/bd/...
1,ফেনী: ঢাকা-চট্রগ্রাম মহাসড়কের ফেনীর মহিপাল বাস...,"আপডেট: ২১৪৩ ঘণ্টা, নভেম্বর ১৯, ২০২৩",https://www.banglanews24.com/national/news/bd/...
2,মৌলভীবাজার: জেলার শ্রীমঙ্গলে দেশিয় পদ্ধতিতে তৈ...,"আপডেট: ০২১৩ ঘণ্টা, নভেম্বর ২০, ২০২৩",https://www.banglanews24.com/national/news/bd/...
3,সিলেট: সিলেটে একটি পোল্ট্রি অ্যান্ড ফিসারিজ কো...,"আপডেট: ০০০৬ ঘণ্টা, নভেম্বর ২০, ২০২৩",https://www.banglanews24.com/national/news/bd/...
4,ঢাকা: নারায়ণগঞ্জের রূপগঞ্জে জাতীয় নিরাপত্তা গো...,"আপডেট: ০০৫৮ ঘণ্টা, নভেম্বর ২০, ২০২৩",https://www.banglanews24.com/national/news/bd/...
5,ব্রাহ্মণবাড়িয়া: জেলায় কনটেইনার ট্রেনের বগি লাই...,"আপডেট: ০০১৭ ঘণ্টা, নভেম্বর ২০, ২০২৩",https://www.banglanews24.com/national/news/bd/...
6,ঢাকা: রাজধানীর ওয়ারী থানার সামনে ককটেল বিস্ফো...,"আপডেট: ২৩৫২ ঘণ্টা, নভেম্বর ১৯, ২০২৩",https://www.banglanews24.com/national/news/bd/...
7,বান্দরবান: বান্দরবানের লামায় জমি চাষের জন্য শ্...,"আপডেট: ২১৩৫ ঘণ্টা, নভেম্বর ১৯, ২০২৩",https://www.banglanews24.com/national/news/bd/...
8,ঢাকা: ভয়াবহ মাদক বুপ্রেনরফিনসহ দুই কারবারিকে র...,"আপডেট: ০১৫৯ ঘণ্টা, নভেম্বর ২০, ২০২৩",https://www.banglanews24.com/national/news/bd/...
9,ঢাকা: রাজধানীর যাত্রাবাড়ী থানার সামনে একটি বা...,"আপডেট: ২২৫০ ঘণ্টা, নভেম্বর ১৯, ২০২৩",https://www.banglanews24.com/national/news/bd/...


In [3]:
print("Total Reviews:",len(data))

Total Reviews: 143642


In [4]:
import pandas as pd
import re
# !pip install pandas numpy torch transformers tensorflow


# Define the process_comments function
def process_text(Text):
    Text = re.sub('[^\u0980-\u09FF]', ' ', str(Text))  
    return Text
  
subset_size = int(len(data) * 0.002)
subset_data = data.head(subset_size) # Take a random 10% subset of the data

subset_data.loc[:, 'Answer'] = ''

# Now, apply the cleaning function to the subset data
subset_data['cleaned'] = subset_data['Text'].apply(process_text)

print("Total Reviews:", len(data))
print("Subset Size:", len(subset_data))
print(subset_data[['Text', 'cleaned', 'Answer']])
print(data.loc[35, 'Text'])

Total Reviews: 143642
Subset Size: 287
                                                  Text  \
0    চাঁদপুর: প্রাচীন বন্দর নগরী ও তিন নদীর মোহনায় ...   
1    ফেনী: ঢাকা-চট্রগ্রাম মহাসড়কের ফেনীর মহিপাল বাস...   
2    মৌলভীবাজার: জেলার শ্রীমঙ্গলে দেশিয় পদ্ধতিতে তৈ...   
3    সিলেট: সিলেটে একটি পোল্ট্রি অ্যান্ড ফিসারিজ কো...   
4    ঢাকা: নারায়ণগঞ্জের রূপগঞ্জে জাতীয় নিরাপত্তা গো...   
..                                                 ...   
282  ঢাকা: গবেষণাপত্রে জালিয়াতির অভিযোগে কোরিয়াভিত্...   
283  ঢাকা: অক্টোবরের ২৮ তারিখ থেকে ১৩ নভেম্বর পর্যন...   
284  ঢাকা: ঢাকা মেডিকেল কলেজ (ঢামেক) হাসপাতালের বহি...   
285  খাগড়াছড়ি: প্রধানমন্ত্রী শেখ হাসিনা খাগড়াছড়ির র...   
286  ঢাকা: আগামী দ্বাদশ জাতীয় সংসদ নির্বাচনের তফসিল...   

                                               cleaned Answer  
0    চাঁদপুর  প্রাচীন বন্দর নগরী ও তিন নদীর মোহনায় ...         
1    ফেনী  ঢাকা চট্রগ্রাম মহাসড়কের ফেনীর মহিপাল বাস...         
2    মৌলভীবাজার  জেলার শ্রীমঙ্গলে দেশিয় পদ্ধতিতে তৈ...         
3    সিল

/tmp/ipykernel_33/3779035115.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data.loc[:, 'Answer'] = ''
/tmp/ipykernel_33/3779035115.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['cleaned'] = subset_data['Text'].apply(process_text)


In [5]:
original_subset_data = subset_data.copy()

subset_data['cleaned'].fillna('', inplace=True)

# Calculate the length of each review and remove reviews with the least words
subset_data['length'] = subset_data['cleaned'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
subset_data = subset_data.loc[subset_data['length'] > 1].reset_index(drop=True)

removed_indices = original_subset_data.index.difference(subset_data.index)

print("Subset Size:", len(subset_data))
print("After Cleaning:")
print("Removed {} Small Reviews".format(len(data) - len(subset_data)))
print("Total Reviews:", len(subset_data))
print(subset_data[['Text', 'cleaned']])


Subset Size: 287
After Cleaning:
Removed 143355 Small Reviews
Total Reviews: 287
                                                  Text  \
0    চাঁদপুর: প্রাচীন বন্দর নগরী ও তিন নদীর মোহনায় ...   
1    ফেনী: ঢাকা-চট্রগ্রাম মহাসড়কের ফেনীর মহিপাল বাস...   
2    মৌলভীবাজার: জেলার শ্রীমঙ্গলে দেশিয় পদ্ধতিতে তৈ...   
3    সিলেট: সিলেটে একটি পোল্ট্রি অ্যান্ড ফিসারিজ কো...   
4    ঢাকা: নারায়ণগঞ্জের রূপগঞ্জে জাতীয় নিরাপত্তা গো...   
..                                                 ...   
282  ঢাকা: গবেষণাপত্রে জালিয়াতির অভিযোগে কোরিয়াভিত্...   
283  ঢাকা: অক্টোবরের ২৮ তারিখ থেকে ১৩ নভেম্বর পর্যন...   
284  ঢাকা: ঢাকা মেডিকেল কলেজ (ঢামেক) হাসপাতালের বহি...   
285  খাগড়াছড়ি: প্রধানমন্ত্রী শেখ হাসিনা খাগড়াছড়ির র...   
286  ঢাকা: আগামী দ্বাদশ জাতীয় সংসদ নির্বাচনের তফসিল...   

                                               cleaned  
0    চাঁদপুর  প্রাচীন বন্দর নগরী ও তিন নদীর মোহনায় ...  
1    ফেনী  ঢাকা চট্রগ্রাম মহাসড়কের ফেনীর মহিপাল বাস...  
2    মৌলভীবাজার  জেলার শ্রীমঙ্গলে দেশিয় পদ্ধতিতে তৈ

/tmp/ipykernel_33/3605439447.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  subset_data['cleaned'].fillna('', inplace=True)
/tmp/ipykernel_33/3605439447.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['cleaned'].fillna('', inplace=True)
/tmp/ipykernel_33/3605439447.py:6: SettingWithCopyWarning: 
A value is trying to be set

In [6]:
import pandas as pd
import pyarrow.parquet as pq
import re
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

bnbert_tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
subset_data['tokens'] = ''

# Tokenize the cleaned text
subset_data['tokens'] = subset_data['cleaned'].apply(lambda x: bnbert_tokenizer.tokenize(x))

# Print the DataFrame with tokenized text
print(subset_data[['cleaned', 'tokens']])
tokens = subset_data['tokens']
print(tokens)
print(subset_data.loc[1, 'tokens'])

# Vectorize the cleaned texts using TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(subset_data['cleaned'])

print("TF-IDF matrix shape:", tfidf_matrix.shape)

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

                                               cleaned  \
0    চাঁদপুর  প্রাচীন বন্দর নগরী ও তিন নদীর মোহনায় ...   
1    ফেনী  ঢাকা চট্রগ্রাম মহাসড়কের ফেনীর মহিপাল বাস...   
2    মৌলভীবাজার  জেলার শ্রীমঙ্গলে দেশিয় পদ্ধতিতে তৈ...   
3    সিলেট  সিলেটে একটি পোল্ট্রি অ্যান্ড ফিসারিজ কো...   
4    ঢাকা  নারায়ণগঞ্জের রূপগঞ্জে জাতীয় নিরাপত্তা গো...   
..                                                 ...   
282  ঢাকা  গবেষণাপত্রে জালিয়াতির অভিযোগে কোরিয়াভিত্...   
283  ঢাকা  অক্টোবরের ২৮ তারিখ থেকে ১৩ নভেম্বর পর্যন...   
284  ঢাকা  ঢাকা মেডিকেল কলেজ  ঢামেক  হাসপাতালের বহি...   
285  খাগড়াছড়ি  প্রধানমন্ত্রী শেখ হাসিনা খাগড়াছড়ির র...   
286  ঢাকা  আগামী দ্বাদশ জাতীয় সংসদ নির্বাচনের তফসিল...   

                                                tokens  
0    [চাদ, ##পর, পরা, ##চীন, বন, ##দর, নগরী, ও, তিন...  
1    [ফেনী, ঢাকা, চট, ##র, ##গরা, ##ম, মহা, ##সড, #...  
2    [[UNK], জেলার, শর, ##ীম, ##ঙ, ##গলে, দেশি, ##য...  
3    [সিলেট, সিলেটে, একটি, পে, ##াল, ##টরি, অ, ##যা...  
4    [ঢাকা, না, ##

In [8]:

import torch
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

#max_length=512 
# Preparing data for model
def prepare_data(context, question, answer_text, tokenizer):
    inputs = tokenizer(question, context, truncation=True, padding='max_length', max_length=100, return_tensors='pt')
    answer_tokens = tokenizer.encode(answer_text, add_special_tokens=False)
    try:
        start_position = inputs.input_ids.squeeze().tolist().index(answer_tokens[0])
    except ValueError:
        return None
    end_position = start_position + len(answer_tokens) - 1
    return {
        'input_ids': inputs['input_ids'].squeeze().tolist(),
        'attention_mask': inputs['attention_mask'].squeeze().tolist(),
        'start_positions': start_position,
        'end_positions': end_position
    }

# Example questions and answers (replace these with actual questions and answers)
questions = ["Example question for the context"] * len(subset_data)  # Replace with actual questions
answers = ["Example answer text"] * len(subset_data)  # Replace with actual answers

# Prepare dataset
dataset = []
for i in range(len(subset_data)):
    context = subset_data['cleaned'].iloc[i]
    question = questions[i]
    answer_text = answers[i]
    prepared_data = prepare_data(context, question, answer_text, bnbert_tokenizer)
    if prepared_data:
        dataset.append(prepared_data)

 

In [9]:
from transformers import TrainingArguments, Trainer

class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])
    
# Split the dataset
train_size = int(0.8 * len(dataset))
train_dataset = dataset[:train_size]
eval_dataset = dataset[train_size:]

train_encodings = {
    'input_ids': [d['input_ids'] for d in train_dataset],
    'attention_mask': [d['attention_mask'] for d in train_dataset],
    'start_positions': [d['start_positions'] for d in train_dataset],
    'end_positions': [d['end_positions'] for d in train_dataset],
}

eval_encodings = {
    'input_ids': [d['input_ids'] for d in eval_dataset],
    'attention_mask': [d['attention_mask'] for d in eval_dataset],
    'start_positions': [d['start_positions'] for d in eval_dataset],
    'end_positions': [d['end_positions'] for d in eval_dataset],
}

train_dataset = QADataset(train_encodings)
eval_dataset = QADataset(eval_encodings)

In [10]:
# Load the pre-trained model
model = AutoModelForQuestionAnswering.from_pretrained("sagorsarker/bangla-bert-base")

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           
    num_train_epochs=8,               
    per_device_train_batch_size=4,    
    per_device_eval_batch_size=4,     
    warmup_steps=500,                 
    weight_decay=0.01,                
    logging_dir='./logs',            
    logging_steps=10,
)

# Create the Trainer
trainer = Trainer(
    model=model,                          
    args=training_args,                   
    train_dataset=train_dataset,          
    eval_dataset=eval_dataset             
)

# Train the model
trainer.train()

model.safetensors:   0%|          | 0.00/660M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste 

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,4.507400


KeyboardInterrupt: 

In [14]:
model.save_pretrained('./fine-tuned-bangla-bert')
bnbert_tokenizer.save_pretrained('./fine-tuned-bangla-bert')

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


('./fine-tuned-bangla-bert/tokenizer_config.json',
 './fine-tuned-bangla-bert/special_tokens_map.json',
 './fine-tuned-bangla-bert/vocab.txt',
 './fine-tuned-bangla-bert/added_tokens.json',
 './fine-tuned-bangla-bert/tokenizer.json')

In [16]:
!pip install rouge-score


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d1f35e6659bf1977076113a4a054a14d1b28efadbbdb08f515ce9a0c421b1d82
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [18]:

from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

In [19]:
model = AutoModelForQuestionAnswering.from_pretrained('./fine-tuned-bangla-bert')
tokenizer = AutoTokenizer.from_pretrained('./fine-tuned-bangla-bert')

def process_text(Text):
    Text = re.sub('[^\u0980-\u09FF]', ' ', str(Text))
    return Text

 
def find_relevant_context(question, tfidf_matrix, vectorizer, data):
    question_tfidf = vectorizer.transform([question])
    cosine_similarities = (tfidf_matrix * question_tfidf.T).toarray()
    most_relevant_index = cosine_similarities.argmax()
    return data.iloc[most_relevant_index]['cleaned']

def get_answer_with_beam_search(question, context, tokenizer, model, beam_size=5, max_length=100):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits.squeeze()
        end_logits = outputs.end_logits.squeeze()
        start_index = start_logits.argmax().item()
        end_index = end_logits.argmax().item()
        relevant_context = context[start_index:end_index]
        inputs = tokenizer(question, relevant_context, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length)
        outputs = model(**inputs)
        start_logits = outputs.start_logits.squeeze()
        end_logits = outputs.end_logits.squeeze()
        topk_start_indices = torch.topk(start_logits, beam_size).indices
        topk_end_indices = torch.topk(end_logits, beam_size).indices
        current_sequences = [(start_index.item(), start_logits[start_index].item(), None) for start_index in topk_start_indices]
        best_score = float('-inf')
        while True:
            all_candidates = []
            for current_index, current_score, prev_index in current_sequences:
                for end_index in topk_end_indices:
                    if end_index.item() < current_index:
                        continue
                    new_score = current_score + end_logits[end_index].item()
                    all_candidates.append((end_index.item(), new_score, current_index))
            if not all_candidates:
                break
            sorted_candidates = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_size]
            max_score = sorted_candidates[0][1] if sorted_candidates else float('-inf')
            if max_score <= best_score:
                break
            best_score = max_score
            current_sequences = sorted_candidates
            best_sequence = max(current_sequences, key=lambda x: x[1])
            best_end_index = best_sequence[0]
            if best_end_index is not None:
                break
        answer_indices = [best_end_index]
        prev_index = best_sequence[2]
        while prev_index is not None:
            answer_indices.append(prev_index)
            prev_index = next((seq[2] for seq in current_sequences if seq[0] == prev_index), None)
        answer_indices.reverse()
        answer_tokens = inputs.input_ids[0][answer_indices[0]:answer_indices[-1] + 1]
        answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    return answer.strip()


# def get_answer_with_beam_search(question, context, beam_size=5, max_length=512):
#     inputs = tokenizer(question, context, return_tensors='pt', truncation=True, padding='max_length', max_length=max_length)
#     with torch.no_grad():
#         outputs = model(**inputs)

#     start_logits = outputs.start_logits.squeeze()
#     end_logits = outputs.end_logits.squeeze()

#     start_indices = torch.topk(start_logits, beam_size).indices
#     end_indices = torch.topk(end_logits, beam_size).indices

#     best_start = None
#     best_end = None
#     best_score = -float('inf')

#     for start_index in start_indices:
#         for end_index in end_indices:
#             if end_index >= start_index:
#                 score = start_logits[start_index] + end_logits[end_index]
#                 if score > best_score:
#                     best_start = start_index
#                     best_end = end_index
#                     best_score = score

#     answer_tokens = inputs.input_ids[0][best_start:best_end + 1]
#     answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

#     return answer.strip()


 

 
 
# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform(subset_data['cleaned'])

# Load evaluation data
# evaluation_data = pd.read_excel('/kaggle/input/eval-dataset/Eval_Dataset.xlsx')

# evaluation_data['Context'] = evaluation_data['Context'].apply(process_text)
# evaluation_data['Question'] = evaluation_data[' Question '].apply(process_text)
# evaluation_data['Answer'] = evaluation_data['Answer'].apply(process_text)


# # Evaluate the model
# em_scores = []
# rouge_scores = []
# bleu_scores = []

# # Initialize ROUGE scorer
# rouge_scorer_inst = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
# smoothie = SmoothingFunction().method4

# for index, row in evaluation_data.iterrows():
#     context = row['Context']
#     question = row['Question']
#     true_answer = row['Answer']

#     # Find relevant context and get the predicted answer
#     relevant_context = find_relevant_context(question, tfidf_matrix, vectorizer, subset_data)
#     predicted_answer = get_answer_with_beam_search(question, relevant_context)

#     # Calculate Exact Match (EM)
#     em_score = 1 if predicted_answer.strip() == true_answer.strip() else 0
#     em_scores.append(em_score)

#     # Calculate ROUGE Score
#     rouge_score = rouge_scorer_inst.score(true_answer, predicted_answer)['rougeL'].fmeasure
#     rouge_scores.append(rouge_score)

#     # Calculate BLEU Score
#     bleu_score = sentence_bleu([true_answer.split()], predicted_answer.split(), smoothing_function=smoothie)
#     bleu_scores.append(bleu_score)

# # Calculate and print the average scores
# if em_scores:
#     avg_em = sum(em_scores) / len(em_scores)
# else:
#     avg_em = 0
    
# if rouge_scores:
#     avg_rouge = sum(rouge_scores) / len(rouge_scores)
# else:
#     avg_rouge = 0
    
# if bleu_scores:
#     avg_bleu = sum(bleu_scores) / len(bleu_scores)
# else:
#     avg_bleu = 0

# print(f"Average EM Score: {avg_em}")
# print(f"Average ROUGE Score: {avg_rouge}")
# print(f"Average BLEU Score: {avg_bleu}")

In [20]:
eval_data = pd.read_excel('/kaggle/input/eval-dataset/Eval_Dataset.xlsx')
# print(eval_data.head())
def process_text(Text):
    Text = re.sub('[^\u0980-\u09FF]', ' ', str(Text))
    return Text

# Apply the cleaning function to the Context column
eval_data['cleaned_context'] = eval_data['Context'].apply(process_text)
eval_data['Answer'].fillna('', inplace=True)

eval_data[' Question '] = eval_data[' Question '].astype(str)
eval_data['cleaned_context'] = eval_data['cleaned_context'].astype(str)

# # tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")

# # # Tokenize the cleaned context
# # eval_data['tokens'] = eval_data['cleaned_context'].apply(lambda x: tokenizer.tokenize(x))

# print(eval_data[['cleaned_context', 'tokens']].head())

/tmp/ipykernel_33/4207749966.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  eval_data['Answer'].fillna('', inplace=True)


In [21]:
!pip install rouge-score



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
pip install pandas transformers torch rouge-score nltk


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install rouge


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [25]:
from sklearn.metrics import accuracy_score
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
model = AutoModelForQuestionAnswering.from_pretrained('./fine-tuned-bangla-bert')

# Define a function to get answers from the model
def get_answer(question, context):
    if not isinstance(question, str) or not isinstance(context, str):
        raise ValueError("Both question and context must be strings.")
    # Tokenize the inputs with truncation to the maximum length of 512
    inputs = tokenizer(question, context, max_length=512, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end]))
    return answer
# Generate predictions
eval_data['predicted_answer'] = eval_data.apply(lambda row: get_answer(row[' Question '], row['cleaned_context']), axis=1)

# Filter out rows where the predicted answer or the actual answer is empty
eval_data_non_empty = eval_data[(eval_data['predicted_answer'].str.strip() != '') & (eval_data['Answer'].str.strip() != '')]

# Display the first few rows of the result
print(eval_data_non_empty[[' Question ', 'cleaned_context', 'Answer', 'predicted_answer']].head())

# Calculate Exact Match (EM) score
def exact_match(pred, true):
    return int(pred == true)

eval_data_non_empty['em'] = eval_data_non_empty.apply(lambda row: exact_match(row['predicted_answer'], row['Answer']), axis=1)
em_score = eval_data_non_empty['em'].mean()
print(f'Exact Match (EM) score: {em_score}')

# Calculate ROUGE score
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(eval_data_non_empty['predicted_answer'].tolist(), eval_data_non_empty['Answer'].tolist(), avg=True)
print(f'ROUGE score: {scores}')

# Calculate BLEU score
from nltk.translate.bleu_score import sentence_bleu

def bleu_score(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    return sentence_bleu(reference, candidate)

eval_data_non_empty['bleu'] = eval_data_non_empty.apply(lambda row: bleu_score(row['Answer'], row['predicted_answer']), axis=1)
bleu_score_avg = eval_data_non_empty['bleu'].mean()
print(f'BLEU score: {bleu_score_avg}')

                                           Question   \
1          কোন ধরনের প্রতিষ্ঠানে আগুন লাগানো হয়েছে?   
2                                 কখন এই ঘটনা ঘটেছে?   
3                      কোন বিদ্যালয়ে এই ঘটনা ঘটেছে?   
5                 কত স্থানে অগ্নিসংযোগের ঘটনা ঘটেছে?   
6  কোন সময়কালের মধ্যে এই অগ্নিসংযোগের ঘটনাগুলি ঘ...   

                                     cleaned_context  \
1  মানিকগঞ্জ মানিকগঞ্জের ঘিওর উপজেলার বালিয়াখোড়া ...   
2  মানিকগঞ্জ  মানিকগঞ্জের ঘিওর উপজেলার বালিয়াখোড়া...   
3  মানিকগঞ্জ মানিকগঞ্জের ঘিওর উপজেলার বালিয়াখোড়া ...   
5  ঢাকা  দ্বাদশ জাতীয় সংসদ নির্বাচনের তফসিল ঘোষণা...   
6  ঢাকা  দ্বাদশ জাতীয় সংসদ নির্বাচনের তফসিল ঘোষণা...   

                                              Answer  \
1  একটি প্রাথমিক বিদ্যালয়ের অফিস কক্ষের পাশের বা...   
2        বৃহস্পতিবার, ১৬ নভেম্বর ভোরে এই ঘটনা ঘটেছে।   
3  ১৫ নম্বর সরকারি প্রাথমিক বিদ্যালয়ে এই ঘটনা ঘট...   
5      দেশজুড়ে ১১টি স্থানে অগ্নিসংযোগের ঘটনা ঘটেছে।   
6  এই অগ্নিসংযোগের ঘটনাগুলি বুধবার, ১৫ নভেম্বর

/tmp/ipykernel_33/1781009302.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data_non_empty['em'] = eval_data_non_empty.apply(lambda row: exact_match(row['predicted_answer'], row['Answer']), axis=1)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Senten

In [23]:
# print(data.loc[220,'Text'])

In [57]:
#Ask a question and get the answer

#question = "চাঁদপুর কত কোটি টাকা ব্যয়ে নির্মাণ করা হয়েছে ৫ কিলোমিটার ড্রেন ?"
#question = "বিএনপি?"
#question = " নারায়ণগঞ্জের রূপগঞ্জে"
#question = "ঢাকা মেট্রোপলিটন পুলিশের  ডিএমপি"
#question = "ঘিওর উপজেলার?"
#question = "মিরিকপুর গ্রাম কোথায়?"
#question = "শাহমাহমুদপুর ইউনিয়ন কোথায়"
#question = "পররাষ্ট্রমন্ত্রী এ কে আব্দুল মোমেন" 
# question = "নীলফামারীর জলঢাকায়" loc[80
#question = "প্রধানমন্ত্রী শেখ হাসিনার"
# question = "জয়পুরহাটের কালাই উপজেলা"  #52
# question = "ফরিদপুরের কানাইপুর?"
#question = "ফিচার লেখক সমুদ্র হক এ এস এম খবিরুল হক?"
#question = "রাজশাহী সিটি মেয়র ?"
#question = "পররাষ্ট্রমন্ত্রী কে?"
question = "রাজধানীতে মাদকবিরোধী অভিযান চালিয়ে কতজনকে  জনকে গ্রেপ্তার করেছে পুলিশ?" 
#question = "রাঙামাটি কোতোয়ালি থানার ভারপ্রাপ্ত কর্মকর্তা" 

#question = "বাংলাদেশ বিশ্ববিদ্যালয় মঞ্জুরি কমিশন (ইউজিসি) এর সদস্য ও ইআরডিএফবির সভাপতি কে?"
#question = "ঢাকা জেলার দোহার বাজারে কোথায় আগুন লেগেছিলো ?"

context = find_relevant_context(question, tfidf_matrix, vectorizer, subset_data)
print("Relevant Answer:", context)

# answer = get_answer_with_beam_search(question, context, tokenizer, model)
# print("Answer:", answer)


Relevant Answer: ঢাকা  রাজধানীতে মাদকবিরোধী অভিযানে বিক্রি ও সেবনের অভিযোগে ২২ জনকে গ্রেপ্তার করেছে ঢাকা মেট্রোপলিটন পুলিশ  ডিএমপি    বুধবার  ১৫ নভেম্বর  সকাল ৬টা থেকে বৃহস্পতিবার  ১৬ নভেম্বর  সকাল ৬টা পর্যন্ত রাজধানীর বিভিন্ন থানা এলাকায় অভিযান চালিয়ে তাদের গ্রেপ্তার করা হয়          ডিএমপির মিডিয়া অ্যান্ড পাবলিক রিলেশনস বিভাগের অতিরিক্ত উপ কমিশনার  এডিসি  কে এন রায় নিয়তি জানান  আসামিদের কাছ থেকে ৮৬৩ পিস ইয়াবা  ১০ কেজি ৫০০ গ্রাম গাঁজা  ২০ গ্রাম হেরোইন ও ১৫টি ট্যাপেন্টাডল ট্যাবলেট জব্দ করা হয়   আসামিদের বিরুদ্ধে ডিএমপির থানায় মাদকদ্রব্য নিয়ন্ত্রণ আইনে ১৯টি মামলা দায়ের করা হয়েছে বলে জানান তিনি   বাংলাদেশ সময়  ১০৪১ ঘণ্টা  নভেম্বর ১৬  ২০২৩ এমএমআই আরআইএস


In [27]:
# print(data.loc[320,'Text'])

In [ ]:
# eval_results = trainer.evaluate()
# print("Evaluation results:", eval_results)

In [ ]:
# eval_results = trainer.evaluate()

# # Print evaluation results
# print("Evaluation results:", eval_results)